# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [2]:
!pip install gradio


In [4]:
# imports

from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import gradio as gr
import ollama

In [6]:
from openai import OpenAI
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [8]:
system_prompt =  "You are a helpful assistant that answers tchnical questions."
system_prompt += "Always be accurate. If you don't know or not sure about some information, say so."

In [12]:
!pip install deep_translator

In [14]:
from deep_translator import GoogleTranslator

# First install deep_translator:
# pip install deep_translator

# Top 10 most spoken languages with their codes
LANGUAGES = {
    "English": "en",
    "Mandarin Chinese": "zh-CN",
    "Hindi": "hi",
    "Spanish": "es",
    "Arabic": "ar",
    "Bengali": "bn",
    "Portuguese": "pt",
    "Russian": "ru",
    "Japanese": "ja",
    "German": "de"
}

In [16]:
def translate_message(text, target_lang):
    if target_lang == "en":
        return text
    try:
        translator = GoogleTranslator(source='auto', target=target_lang)
        return translator.translate(text)
    except:
        return f"Translation error: {text}"

In [18]:
def chat(message, history):
    # Original chat processing
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}] 
    response = openai.chat.completions.create(model = MODEL, messages = messages)
    response_text = response.choices[0].message.content
    
    if chat_state.speak:
        talker(response_text)
    
    # Translate messages
    translated_message = translate_message(message, chat_state.target_lang)
    translated_response = translate_message(response_text, chat_state.target_lang)
    
    gr.Chatbot.update(value=[(translated_message, translated_response)], visible=True)
    
    return response_text

In [20]:
# allows you to convert text into speech using Google's Text-to-Speech API
!pip install gTTS

In [21]:
from gtts import gTTS
import os
import tempfile

def text_to_speech(text, lang_code):
    try:
        tts = gTTS(text=text, lang=lang_code)
        # Create temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as fp:
            tts.save(fp.name)
            return fp.name
    except:
        return None

In [24]:
    def respond(message, history):
        bot_response, history_original, history_translated = process_message(
            message, 
            history, 
            'translated' if speech_language.value.lower() == 'translated' else 'original'
        )
        return "", history_original, history_translated

In [26]:
def process_message(message, history, speech_mode):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}] 
    response = openai.chat.completions.create(model = MODEL, messages = messages)
    response_text = response.choices[0].message.content
    
    # Create audio if speech is enabled
    audio_path = None
    if chat_state.speak:
        if speech_mode == "Translated":
            translated_text = translate_message(response_text, chat_state.target_lang)
            audio_path = text_to_speech(translated_text, chat_state.target_lang)
        else:
            talker(response_text)
    
    # Translate messages for display
    translated_message = translate_message(message, chat_state.target_lang)
    translated_response = translate_message(response_text, chat_state.target_lang)
    
    history_original = history + [
        {"role": "user", "content": message},
        {"role": "assistant", "content": response_text}
    ]
    history_translated = [
        {"role": "user", "content": translated_message},
        {"role": "assistant", "content": translated_response}
    ]
    
    return response_text, history_original, history_translated, audio_path

with gr.Blocks() as demo:
    speech_mode = gr.State("Original")
    
    with gr.Row():
        speak_checkbox = gr.Checkbox(
            label="Read responses aloud",
            value=True,
            interactive=True
        )
        language_dropdown = gr.Dropdown(
            choices=list(LANGUAGES.keys()),
            value="Spanish",
            label="Translation Language",
            interactive=True
        )
        speech_language = gr.Radio(
            choices=["Original", "Translated"],
            value="Original",
            label="Speech Language",
            interactive=True
        )
    
    # Add audio player
    audio_player = gr.Audio(label="Response Audio", visible=True)
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Original Conversation")
            chatbot_original = gr.Chatbot(type="messages")
            msg_original = gr.Textbox(label="Message")
            send_btn = gr.Button("Send")
        
        with gr.Column():
            gr.Markdown("### Translated Conversation")
            chatbot_translated = gr.Chatbot(type="messages")
    
    state = gr.State([])
    
    def respond(message, history, current_speech_mode):
        bot_response, history_original, history_translated, audio_path = process_message(
            message, 
            history,
            current_speech_mode
        )
        
        return "", history_original, history_translated, audio_path
    
    send_btn.click(
        respond,
        inputs=[msg_original, state, speech_language],
        outputs=[msg_original, chatbot_original, chatbot_translated, audio_player],
    )
    
    msg_original.submit(
        respond,
        inputs=[msg_original, state, speech_language],
        outputs=[msg_original, chatbot_original, chatbot_translated, audio_player],
    )
    
    speak_checkbox.change(fn=lambda x: setattr(chat_state, 'speak', x), inputs=[speak_checkbox])

demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
